In [212]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [125]:
url = '''
https://search.naver.com/search.naver?
where=news&sm=tab_pge&
query={search_word}
&sort=0
&photo=0&field=0&pd=3&
ds={date}&de={date}
&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,
p:from{date_}to{date_},
a:all&
start={page}
'''

date = '2022.04.19'
date_ = date.replace('.','')
search_word = '사회적 거리두기'
search_word = search_word.replace(' ','+')
url = ''.join(url.format(date=date, date_ = date_, search_word=search_word ,page=4444).split('\n'))
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
soup_a = soup.select('a.btn')
''.join(re.findall('[0-9]',str(soup_a[-1]).split(' ')[-1]))
# a=[i['href'] for i in soup_a]
# b=[i['title'] for i in soup_a]
# c=['2020' ]* len(a)
# c

'167'

In [141]:
url = ''.join('''
https://search.naver.com/search.naver?
where=news&sm=tab_pge&
query={search_keyword}
&sort=0
&photo=0&field=0&pd=3&
ds={date}&de={date}
&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,
p:from{date_}to{date_},
a:all&
start={page}
'''.split('\n'))

print(url)

https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}


In [372]:
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y.%m.%d") for date in pandas.date_range(start, periods=(end-start).days+1)]
    return dates

def make_soup(url):
    response = requests.get(url_)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def naver_news_url_crawling(search_keyword,startdate='20220101',enddate='20220101'):

    
    search_keyword = search_keyword.replace(' ','+')
    
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'
    
    dates = date_range(startdate,enddate)
    result = pd.DataFrame(columns=['date','title','url'])
    for date in dates:
        
        url_ = url.format(search_keyword = search_keyword,
                          date = date,
                          date_ = date.replace('.',''),
                          page=4000)
        
        soup = make_soup(url_).select('a.btn')


        botton_count = int(re.findall('[0-9]+',str(soup[-1]).split(' ')[-1])[0]) * 10 - 9

        
        for page in range(1,botton_count+1,10):
            
            url_ = url.format(search_keyword = search_keyword,
                              date = date,
                              date_= date.replace('.',''),
                              page = page)

            soup = make_soup(url_).select('a.news_tit')
            
            a_page_ulr = [html['href'] for html in soup]
            a_page_title = [html['title'] for html in soup]
            a_page_date = [date] * len(a_page_title)
            
            result= pd.concat([result , pd.DataFrame({'date':a_page_date, 'title':a_page_title, 'url':a_page_ulr})])
            

    result['news_agency'] = result['url'].apply(lambda x : x.split('/')[2])
    return result

In [397]:
df = naver_news_url_crawling('사회적 거리두기',startdate='20220101',enddate='20220101')
df

,date,title,url,news_agency
0,2022.01.01,"[속보]거리두기 2주 연장…4인 집합금지·식당,카페 9시 영업제한 Q&A 정리",http://www.gukjenews.com/news/articleView.html?idxno=2380605,www.gukjenews.com
1,2022.01.01,"전남도, 사회적 거리두기 16일까지 연장",http://www.fnnews.com/news/202201011138320972,www.fnnews.com
2,2022.01.01,"'거리두기 반복' 험난한 일상회복의 길… ""앞으로 3년은 더""",http://moneys.mt.co.kr/news/mwView.php?no=2021123115538071771,moneys.mt.co.kr
3,2022.01.01,[사회in]거리두기 2주 '연장'…연초에도 자영업 한숨 계속,http://www.edaily.co.kr/news/newspath.asp?newsid=01341526632193456,www.edaily.co.kr
4,2022.01.01,‘4인·9시’ 거리두기 2주 연장… 10일부터 백화점·마트도 방역패스,http://www.segye.com/content/html/2021/12/31/20211231512062.html?OutUrl=naver,www.segye.com
...,...,...,...,...
3,2022.01.01,변광용 경남 거제시장 신년사,https://www.viva100.com/main/view.php?key=20211227010007561,www.viva100.com
4,2022.01.01,"<신년인터뷰> 이태환 세종시 의장 ""국회 세종의사당 건립에 최선”",https://www.viva100.com/main/view.php?key=20220101010000024,www.viva100.com
5,2022.01.01,"세종충남대병원, 종무식서 ""지역민 건강 책임 소명 다할 것""",https://www.viva100.com/main/view.php?key=20220101010000011,www.viva100.com
6,2022.01.01,"[영상] ""어서 호랑이~""...임인년 '언택트 해돋이' 생중계, 일출시간은?",https://www.onews.tv/news/articleView.html?idxno=105773,www.onews.tv


In [399]:
a = (df.groupby('news_agency').count()['date'].sort_values(ascending=False) > 10).reset_index()
a[a.date == True]

,news_agency,date
0,www.news1.kr,True
1,www.newsis.com,True
2,www.bzeronews.com,True
3,yna.kr,True
4,view.asiae.co.kr,True
5,www.viva100.com,True
6,www.kmaeil.com,True


In [268]:
a = 'http://moneys.mt.co.kr/news/mwView.php?no=2022010110548022148'
response = requests.get(a)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
b = str(soup.select('#_article div'))
def main_text_crawling(df):
    return None

"[<>핫한 SNS 치킨맛집 '이유치킨'이 최근 서울 방이점을 새롭게 오픈했다. 이유치킨의 가장 큰 장점은 독특한 메뉴 가성비다. 치킨을 신선한 기름에 튀기고 오븐에 굽는 두 번의 조리 과정으로 치킨의 풍미를 살렸다.\xa0</>, <></></>, <>육즙을 살린 베이크 치킨과 바삭한 식감의 크리스피 치킨으로 나눠 소비자 선택의 폭을 넓혔다. 가성비는 물론 인테리어와 패키지 디자인도 더욱 고급스럽게 신경썼다. 공격적인 오픈 이벤트와 맛과 가격을 모두 잡은 ‘갓성비’ 메뉴로 소비자 만족도를 더욱 높이고 있다.</>, <></></>, <>이유치킨의 대표적인 시그니처 메뉴인 ‘파스타치요’는 ‘파스타와 치킨이 만난 요리’라는 뜻의 파스타치요는 치킨과 치즈오븐 파스타를 합리적인 가격에 제공하고 있다. 이외에도 블랙갈릭, 메이플허니, 홍차오, 청양마요, 김치킹 등 다채로운 치킨과 사이드를 구비하고 있다.</>, <></></>, <>한편, 치킨 프랜차이즈 이유치킨은 브랜드변경(업종변경)시 창업자금을 지원해주고 있다.</>]"

In [371]:
df.news_agency == 'moneys.mt.co.kr'

0    True 
1    False
2    False
3    False
4    False
     ...  
5    False
6    False
7    False
8    False
9    False
Name: news_agency, Length: 204, dtype: bool